# Setting Up Network

## Assigning a reference location and random weights

This script generates randomized weight distributions (`wt_1000` to `wt_6000`) for 300 active zones (excluding 100 zero-weight zones), capped at 25 lbs each, and saves them in `Centroids.csv`. A fixed access point (ID 9999) is added. Randomization ensures varied but controlled loads per scenario.

In [3]:
from dbfread import DBF
import pandas as pd
import numpy as np

# loading centroids
centroids_dbf = DBF("Centroids.dbf")
centroids_df = pd.DataFrame(iter(centroids_dbf))

# Add reference centroid (access point) with ID 9999
ref = pd.DataFrame([{'cid': 9999, 'x': 500, 'y': 400}])
centroids_df = pd.concat([centroids_df, ref], ignore_index=True)

def assign_random_weights(df, total_weight):
    np.random.seed()  # new random draw each run
    weight_col = f'wt_{total_weight}'
    df[weight_col] = 0.0

    # Eligible nodes (exclude depot)
    valid_idx = df[df['cid'] != 9999].index

    # Pick 100 zones with zero weight
    zero_idx = np.random.choice(valid_idx, 100, replace=False)
    active_idx = valid_idx.difference(zero_idx)

    # Assign initial random weights (1–25)
    weights = np.random.randint(1, 26, size=len(active_idx)).astype(float)

    # Scale iteratively to match total_weight (without exceeding 25)
    while abs(weights.sum() - total_weight) > 0.01:
        diff = total_weight - weights.sum()
        if diff > 0:
            # Need to add weight
            idx = np.random.choice(len(weights))
            if weights[idx] < 25:
                add = min(diff, 25 - weights[idx], 1)  # add at most 1 lb per step
                weights[idx] += add
        elif diff < 0:
            # Need to subtract weight
            idx = np.random.choice(len(weights))
            if weights[idx] > 1:
                sub = min(-diff, weights[idx] - 1, 1)  # subtract at most 1 lb per step
                weights[idx] -= sub

    # Assign final weights back
    df.loc[active_idx, weight_col] = weights
    df.loc[df['cid'] == 9999, weight_col] = 0
    return df

# Run for multiple caps
for w in [1000, 2000, 3000, 4000, 5000, 6000]:
    centroids_df = assign_random_weights(centroids_df, w)

# Save to CSV
centroids_df.to_csv("Centroids.csv", index=False)


## Generating Filtered Link Networks for Each Weight Scenario

This script filters the full link network (`Drone_Links_AllToAll.csv`) for each weight scenario (`wt_1000` to `wt_6000`) by retaining only the 20 nearest neighbors per active node and all depot-related links. It ensures bidirectionality and assigns unique IDs to depot edges. Outputs are saved as `Drone_Links_Filtered_*.csv`.

In [11]:
import pandas as pd
import numpy as np

# Load centroids and links
centroids_df = pd.read_csv("Centroids.csv")
links_df = pd.read_csv("Drone_Links_AllToAll.csv")

# Weight scenarios
weight_columns = ["wt_1000", "wt_2000", "wt_3000", "wt_4000", "wt_5000", "wt_6000"]
k_neighbors = 20  # number of nearest neighbors per node

for wcol in weight_columns:
    # Active nodes (non-zero weight) + depot
    weights = dict(zip(centroids_df['cid'], centroids_df[wcol] * 0.453592))
    active_nodes = [cid for cid, w in weights.items() if w > 0.0]
    nodes = active_nodes + [9999]  # include depot

    # Filter links to just these nodes
    sublinks = links_df[(links_df['from_id'].isin(nodes)) & (links_df['to_id'].isin(nodes))]

    # For each node, find up to 20 nearest neighbors (excluding depot)
    filtered_rows = []
    for node in active_nodes:
        nbrs = sublinks[(sublinks['from_id'] == node) & (sublinks['to_id'].isin(active_nodes))]
        nbrs = nbrs.sort_values('length').head(k_neighbors)
        filtered_rows.append(nbrs)

    # Combine all selected neighbor edges
    filtered_df = pd.concat(filtered_rows, ignore_index=True)

    # --- Make the network bidirectional ---
    # For every (a,b), ensure (b,a) is present
    reverse_edges = filtered_df.rename(columns={'from_id': 'to_id', 'to_id': 'from_id'}).copy()
    # Merge original and reverse edges
    filtered_df = pd.concat([filtered_df, reverse_edges], ignore_index=True)
    # Drop duplicates (same direction pairs kept only once)
    filtered_df.drop_duplicates(subset=['from_id', 'to_id'], inplace=True)

    # Always keep depot edges (both directions) from sublinks
    depot_links = sublinks[(sublinks['from_id'] == 9999) | (sublinks['to_id'] == 9999)]
    filtered_df = pd.concat([filtered_df, depot_links], ignore_index=True)
    filtered_df.drop_duplicates(subset=['from_id', 'to_id'], inplace=True)

    # --- Assign unique IDs for depot-related links (starting at 200001) ---
    max_existing_id = links_df['id'].max() if 'id' in links_df.columns else 0
    next_id = max(200001, max_existing_id + 1)

    depot_mask = (filtered_df['from_id'] == 9999) | (filtered_df['to_id'] == 9999)
    depot_count = depot_mask.sum()
    new_ids = range(next_id, next_id + depot_count)
    filtered_df.loc[depot_mask, 'id'] = new_ids

    # Save filtered dataset
    out_file = f"Drone_Links_Filtered_{wcol.split('_')[1]}.csv"
    filtered_df.to_csv(out_file, index=False)
    print(f"Saved {out_file} with {len(filtered_df)} bidirectional edges "
          f"for {len(active_nodes)} active nodes (k={k_neighbors}). "
          f"Depot links assigned IDs starting at {next_id}.")


Saved Drone_Links_Filtered_1000.csv with 12180 bidirectional edges for 300 active nodes (k=20). Depot links assigned IDs starting at 200001.
Saved Drone_Links_Filtered_2000.csv with 12180 bidirectional edges for 300 active nodes (k=20). Depot links assigned IDs starting at 200001.
Saved Drone_Links_Filtered_3000.csv with 12180 bidirectional edges for 300 active nodes (k=20). Depot links assigned IDs starting at 200001.
Saved Drone_Links_Filtered_4000.csv with 12180 bidirectional edges for 300 active nodes (k=20). Depot links assigned IDs starting at 200001.
Saved Drone_Links_Filtered_5000.csv with 12180 bidirectional edges for 300 active nodes (k=20). Depot links assigned IDs starting at 200001.
Saved Drone_Links_Filtered_6000.csv with 12180 bidirectional edges for 300 active nodes (k=20). Depot links assigned IDs starting at 200001.


## Adding Depot Links to the Truck Network

This script loads `Grid_Lines_Melted_v2.dbf`, appends four custom links to connect the depot (ID 9999) to the network, and saves the updated truck path dataset as `Grid_Lines_Melted.csv`. These links ensure connectivity for depot access in routing models.

In [16]:
from dbfread import DBF
import pandas as pd
import numpy as np

# loading centroids
truck_dbf = DBF("Grid_Lines_Melted_v2.dbf")
truck_df = pd.DataFrame(iter(truck_dbf))

#Adding the two links between depot and network edge
link_1 = pd.DataFrame([{'line_id': 9999, 'x1': 400, 'y1': 400, 'x2': 500, 'y2': 400, 'Truck_Path': 1, 'length': 100, 'cid': 9999, 'int_line_i': 419}])
link_2 = pd.DataFrame([{'line_id': 9999, 'x1': 400, 'y1': 400, 'x2': 500, 'y2': 400, 'Truck_Path': 1, 'length': 100, 'cid': 9999, 'int_line_i': 839}])
link_3 = pd.DataFrame([{'line_id': 419, 'x1': 380, 'y1': 400, 'x2': 400, 'y2': 400, 'Truck_Path': 1, 'length': 20, 'cid': 399, 'int_line_i': 9999}])
link_4 = pd.DataFrame([{'line_id': 839, 'x1': 400, 'y1': 380, 'x2': 400, 'y2': 400, 'Truck_Path': 1, 'length': 20, 'cid': 399, 'int_line_i': 9999}])
truck_df = pd.concat([truck_df, link_1, link_2, link_3, link_4], ignore_index=True)

# Save to CSV
truck_df.to_csv("Grid_Lines_Melted.csv", index=False)